# Re Run Step Function Executions

### Imports & Creds :

In [1]:
import boto3
import datetime
from dateutil import tz
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create input widgets
state_machine_input = widgets.Text(description="State Machine Name:", placeholder="e.g., RAPISAMStateMachine-1YwcU3XmIcZ5")
supplier_id_input = widgets.Text(description="Supplier ID:", placeholder="e.g., SAGREETINGS")
start_time_input = widgets.Text(description="Start Time UTC:", placeholder="YYYY-MM-DD HH:MM:SS")
run_button = widgets.Button(description="Run", button_style="success")
output = widgets.Output()

# boto3 client for Step Functions (no profile input here, uses default)
stepfunctions = boto3.client('stepfunctions')

### Functons :

In [2]:
def parse_datetime(dt_str):
    try:
        # Parse string to datetime with UTC timezone
        return datetime.datetime.strptime(dt_str, '%Y-%m-%d %H:%M:%S').replace(tzinfo=datetime.timezone.utc)
    except Exception as e:
        raise ValueError(f"Invalid datetime format. Use YYYY-MM-DD HH:MM:SS. Error: {e}")

def get_state_machine_arn(name):
    output.append_stdout(f"[INFO] Looking up state machine ARN for '{name}'...\n")
    paginator = stepfunctions.get_paginator('list_state_machines')
    for page in paginator.paginate():
        for sm in page['stateMachines']:
            if sm['name'] == name:
                output.append_stdout(f"[INFO] Found ARN: {sm['stateMachineArn']}\n")
                return sm['stateMachineArn']
    raise ValueError(f"State machine '{name}' not found.")

def list_supplier_executions(state_machine_arn, supplier_id, start_time, end_time):
    output.append_stdout(f"[INFO] Listing executions for supplier '{supplier_id}' between {start_time} and {end_time}...\n")

    executions = []
    paginator = stepfunctions.get_paginator('list_executions')
    total_scanned = 0
    matched_count = 0

    for page in paginator.paginate(stateMachineArn=state_machine_arn):
        for exe in page['executions']:
            total_scanned += 1

            # Stop if execution is older than start_time (pagination in DESC order)
            if exe['startDate'] < start_time:
                output.append_stdout("\n[INFO] Reached older executions. Stopping pagination.\n")
                output.append_stdout(f"[INFO] Total executions scanned: {total_scanned}\n")
                output.append_stdout(f"[INFO] Matching executions found: {matched_count}\n")
                return executions

            # Show live scan progress on one line
            sys.stdout.write(f"\r[SCAN] Scanned: {total_scanned} | Matches: {matched_count}")
            sys.stdout.flush()

            if supplier_id in exe['name'] and start_time <= exe['startDate'] <= end_time:
                matched_count += 1
                executions.append(exe)

    print()  # newline after progress bar
    output.append_stdout(f"[INFO] Total executions scanned: {total_scanned}\n")
    output.append_stdout(f"[INFO] Matching executions found: {matched_count}\n")
    return executions

def get_execution_input(execution_arn):
    output.append_stdout(f"[INFO] Fetching input for execution ARN: {execution_arn}\n")
    response = stepfunctions.describe_execution(executionArn=execution_arn)
    return response['input']

def run_button_clicked(b):
    with output:
        clear_output()
        try:
            # Read inputs
            state_machine_name = state_machine_input.value.strip()
            supplier_id = supplier_id_input.value.strip()
            start_time_str = start_time_input.value.strip()

            # Validate start time input
            start_time_utc = parse_datetime(start_time_str)
            now_utc = datetime.datetime.utcnow().replace(tzinfo=datetime.timezone.utc)

            # Run main logic
            output.append_stdout("[INFO] Script started.\n")
            state_machine_arn = get_state_machine_arn(state_machine_name)
            executions = list_supplier_executions(state_machine_arn, supplier_id, start_time_utc, now_utc)

            if not executions:
                output.append_stdout("[INFO] No matching executions found.\n")
                return

            # Optionally, display execution ARNs found
            output.append_stdout(f"\n[INFO] Matching executions ({len(executions)}):\n")
            for exe in executions:
                output.append_stdout(f"  - {exe['executionArn']} started at {exe['startDate']}\n")

            output.append_stdout("\n[INFO] Script finished.\n")

        except Exception as e:
            output.append_stdout(f"[ERROR] {e}\n")

# Bind button click event
run_button.on_click(run_button_clicked)

### Re Run Executions

In [ ]:
# Display widgets
display(widgets.HTML("<b>Enter Step Functions Query Parameters:</b>"))
display(state_machine_input, supplier_id_input, start_time_input, run_button, output)